In [1]:
import pandas as pd
import numpy as np

# Plotting imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors, hex_to_rgb

import plotly.io as pio
pio.templates.default = "none"
font = dict(family='Computer Modern', size=18)

from ipywidgets import interact, interactive
import ipywidgets as widgets
from IPython.display import display

colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

def scaler(s):
    return (s - max(0, s.min())) / (s.max() - s.min())

In [2]:
df = pd.read_pickle('test_csp_pv.df.gz')

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
tower__cycle_capacity_kw,77.0,1.250000e+05,0.000000e+00,1.250000e+05,1.250000e+05,1.250000e+05,1.250000e+05,1.250000e+05
tower__solar_multiple,77.0,2.353829e+00,6.048340e-01,1.500000e+00,1.750000e+00,2.416667e+00,2.875000e+00,3.437500e+00
tower__tes_hours,77.0,1.086709e+01,2.631420e+00,6.000000e+00,8.916667e+00,1.100000e+01,1.287500e+01,1.600000e+01
pv__system_capacity_kw,77.0,1.150628e+05,5.203835e+04,2.500000e+04,7.421875e+04,1.125000e+05,1.635417e+05,1.927083e+05
pv__tilt,77.0,3.502081e+01,1.200216e+01,1.500000e+01,2.625000e+01,3.750000e+01,4.312500e+01,5.812500e+01
annual_energies__pv,77.0,1.885611e+08,8.581688e+07,4.106261e+07,1.198818e+08,1.866777e+08,2.681503e+08,3.242375e+08
annual_energies__tower,77.0,4.838551e+06,3.614687e+06,2.413919e+06,3.052561e+06,3.619211e+06,3.919313e+06,1.795824e+07
annual_energies__hybrid,77.0,1.918329e+08,8.595102e+07,4.315731e+07,1.226064e+08,1.904458e+08,2.718132e+08,3.282824e+08
internal_rate_of_returns__pv,77.0,-6.962786e+01,7.011282e-02,-6.983413e+01,-6.966363e+01,-6.960581e+01,-6.957971e+01,-6.956143e+01
internal_rate_of_returns__tower,77.0,-7.190052e+01,3.142613e-02,-7.194566e+01,-7.192507e+01,-7.190867e+01,-7.186809e+01,-7.182495e+01


In [27]:
color_by = 'tower__tes_hours'
y_var = 'benefit_cost_ratios__hybrid'

def color_violin(color_by, y_var):
    vals = sorted(pd.unique(df[color_by]))
    ncolors = n_colors('rgb(31, 119, 180)', 'rgb(255, 127, 14)', len(vals), colortype='rgb')
    y = df[y_var]

    fig = go.Figure()

    for val, color in zip(vals, ncolors):
        y_vals = y[df[color_by] == val]
        fig.add_trace(go.Violin(x=y_vals, line_color=color, name=f"{val} {color_by}", points='all'))

    fig.update_xaxes(title=y_var)
    fig.update_yaxes(title=color_by, showticklabels=False)

    fig.update_layout(font=font)
    fig.show()
    

paramList = list(df.columns)

# Drop down to view a specific solution and constraint
selector1 = widgets.Dropdown(options=paramList, value='tower__tes_hours', description='Color By', disabled=False)
selector2 = widgets.Dropdown(options=paramList, value='annual_energies__hybrid', description='X Var', disabled=False)
selector3 = widgets.Dropdown(options=paramList, value='benefit_cost_ratios__hybrid', description='Y Var', disabled=False)

# Create an interactive object
ui = interactive(color_violin, color_by=selector1, x_var=selector2, y_var=selector3)

# Force horizontal layout of controls, with output below
controls = widgets.HBox(ui.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
display(widgets.VBox([controls, ui.children[-1]]))

In [21]:
color_by = 'tower__tes_hours'
x_var = 'annual_energies__hybrid'
y_var = 'benefit_cost_ratios__hybrid'

def color_scatter(color_by, x_var, y_var):
    vals = sorted(pd.unique(df[color_by]))
    ncolors = n_colors('rgb(31, 119, 180)', 'rgb(255, 127, 14)', len(vals), colortype='rgb')
    x_norm = scaler(df[x_var])
    y_norm = scaler(df[y_var])

    fig = go.Figure()

    for val, color in zip(vals, ncolors):
        fig.add_trace(go.Scatter(x=x_norm[df[color_by] == val], y=y_norm[df[color_by] == val],
                                 mode='markers', name=f"{val} {color_by}", marker=dict(color=color)))

    fig.update_xaxes(title=x_var)
    fig.update_yaxes(title=y_var)

    fig.update_layout(font=font)
    fig.show()
    

paramList = list(df.columns)

# Drop down to view a specific solution and constraint
selector1 = widgets.Dropdown(options=paramList, value='tower__tes_hours', description='Color By', disabled=False)
selector2 = widgets.Dropdown(options=paramList, value='annual_energies__hybrid', description='X Var', disabled=False)
selector3 = widgets.Dropdown(options=paramList, value='benefit_cost_ratios__hybrid', description='Y Var', disabled=False)

# Create an interactive object
ui = interactive(color_scatter, color_by=selector1, x_var=selector2, y_var=selector3)

# Force horizontal layout of controls, with output below
controls = widgets.HBox(ui.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
display(widgets.VBox([controls, ui.children[-1]]))

In [35]:
color_by = 'tower__tes_hours'
x_var = 'annual_energies__hybrid'
y_var = 'benefit_cost_ratios__hybrid'

vals = sorted(pd.unique(df[color_by]))
ncolors = n_colors('rgb(31, 119, 180)', 'rgb(255, 127, 14)', len(vals), colortype='rgb')
x_norm = scaler(df[x_var])
y_norm = scaler(df[y_var])

fig = go.Figure()

for val, color in zip(vals, ncolors):
    fig.add_trace(go.Scatter(x=x_norm[df[color_by] == val], y=y_norm[df[color_by] == val],
                             mode='markers', name=f"{val:.0f} Hours TES", marker=dict(color=color)))

fig.update_xaxes(title='Scaled Annual Energy')
fig.update_yaxes(title='Scaled Benefit to Cost Ratio')

fig.update_layout(font=font)
fig.show()

In [36]:
week = 0
hours = 8760

data= df[df['benefit_cost_ratios__hybrid'] == df['benefit_cost_ratios__hybrid'].max()]
vals = ['generation_profile__pv', 'generation_profile__tower']
sides = ['negative', 'positive']
ncolors = [colors[0], colors[1]]
names = ['PV', 'CSP']
bounds = [(10, 44), (61, 112), (129, 163)]

mult = np.array(data['dispatch_factors'].iloc[0])
scaled_mult = scaler(mult)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('High Importance (1-3)', 'Medium Importance (4-7)', 'Low Importance (8-10)'))


for val, side, color, name in zip(vals, sides, ncolors, names):
    y_vals = scaler(np.array(data[val].iloc[0]))
    
    for i, bound in enumerate(bounds):
        lower, upper = bound
        idx = np.logical_and(mult>=lower, mult<=upper)
        
        fig.add_trace(go.Histogram(x=y_vals[idx], histnorm='probability', 
                                   marker_color=color, bingroup=0, opacity=0.75, xbins=dict(size=0.1),
                                   name=name, showlegend=(i==0)), row=3-i, col=1)
        fig.update_yaxes(range=[0,0.8], row=3-i, col=1)

fig.update_xaxes(title='Scaled Energy Production', row=3, col=1)
fig.update_yaxes(title='Fraction of Periods', row=2, col=1)

fig.update_layout(bargap=0.1, showlegend=True, font=font)
fig.show()
print(data[list(data.columns)[:5]])

     tower__cycle_capacity_kw  tower__solar_multiple  tower__tes_hours  \
486                  125000.0                    2.7              14.0   

     pv__system_capacity_kw  pv__tilt  
486                200000.0     26.25  


In [40]:

data= df[df['lcoe_nom__hybrid'] == df['lcoe_nom__hybrid'].min()]
vals = ['generation_profile__pv', 'generation_profile__tower', 'dispatch_factors']
sides = ['negative', 'positive', None]
ncolors = [colors[0], colors[1], colors[2]]
names = ['PV', 'CSP', 'Price']
weeks = [1, 26, 35]
hours = 24 * 4
offset = 24 * 3

mult = np.array(data['dispatch_factors'].iloc[0])
scaled_mult = scaler(mult)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('January', 'July', 'September'))


for val, side, color, name in zip(vals, sides, ncolors, names):
    y_vals = scaler(np.array(data[val].iloc[0]))
    
    for i, week in enumerate(weeks):
        
        fig.add_trace(go.Scatter(x=np.arange(hours), y=y_vals[offset+week*168:offset+week*168+hours], mode='lines',
                                   name=name, showlegend=(i==0), line_color=color), row=3-i, col=1)
        fig.update_yaxes(range=[0,1.05], row=3-i, col=1)

fig.update_xaxes(title='Hours', row=3, col=1)
fig.update_yaxes(title='Scaled Energy Production', row=2, col=1)

fig.update_layout(showlegend=True, font=font)
fig.show()
print(data[list(data.columns)[:5]])

     tower__cycle_capacity_kw  tower__solar_multiple  tower__tes_hours  \
481                  125000.0                    2.7              12.0   

     pv__system_capacity_kw  pv__tilt  
481                200000.0     26.25  


In [181]:
week = 0
hours = 8760

data= df[df['benefit_cost_ratios__hybrid'] == df['benefit_cost_ratios__hybrid'].max()]
vals = ['generation_profile__pv', 'generation_profile__tower']
sides = ['negative', 'positive']
ncolors = ['blue', 'orange']
names = ['PV', 'CSP']
bounds = [(10, 44), (61, 112), (129, 163)]

mult = np.array(data['dispatch_factors'].iloc[0])

fig = go.Figure()

for val, side, color, name in zip(vals, sides, ncolors, names):
    y_vals = scaler(np.array(data[val].iloc[0]))
    for i, bound in enumerate(bounds):
        lower, upper = bound
        idx = np.logical_and(mult>=lower, mult<=upper)
        
        fig.add_trace(go.Violin(x=i*np.ones(len(y_vals)), y=y_vals[idx], side=side, orientation='v', line_color=color,
                                legendgroup=name, name=name, showlegend=False, meanline_visible=True))

fig.update_xaxes(title='Normalized Load Importance')
fig.update_yaxes(title='Normalized Annual Energy')

fig.update_layout(violinmode='overlay', violingap=0)
fig.show()
print(data[list(data.columns)[:5]])

     tower__cycle_capacity_kw  tower__solar_multiple  tower__tes_hours  \
486                  125000.0                    2.7              14.0   

     pv__system_capacity_kw  pv__tilt  
486                200000.0     26.25  
